# FIT3182 - Big data management and processing

Name: Cheong Karr Kei

Student ID: 30091497

Email: kche0070@student.monash.edu


# Assignment Part A #

**Task 1. MongoDB Data Model**


Based on the two data sets provided (`hotspot_historic.csv` and `climate_historic.csv`), a suitable data model to support the efficient querying of the two data sets would be a `One-to-Many Relationship Model` by using `Document References`. 

According to the specifications, climate data is recorded on a daily basis whereas fire data is recorded based on the occurrence of a fire on a particular day. This means that for one climate data, there can be zero or many fire data, thus the one-to-many relationship. 

Let us look at an example of this model using the two data sets provided. 

In [1]:
import pymongo
from pymongo import MongoClient

# Connect on the default host and port
client = MongoClient () 

#Create/get database 
db = client.taskA_db 

To create this example, we will look at the entries on `27th December 2018` in both datasets. In `climate_historic.csv`, there is only one entry where:
- station: 948702 
- date: 27/12/18
- air_temperature_celcius: 28
- relative_humidity: 58.3 
- windspeed_knots: 9.3 
- max_wind_speed: 15.9 
- precipitation: 0.00I
- GHI_w/m2: 223

Let us create a new document, note that we change the date to 27/12/2018 format to match with the dates in `hotspot_historic.csv` for demonstration purposes

In [2]:
#Create/get collection for climates
climates = db.climates
#clear collection
climates.drop()

In [3]:
newClimate = {
    "station": 948702,
    "date": "27/12/2018",
    "air_temperature_celcius": 28,
    "relative_humidity": 58.3,
    "windspeed_knots": 9.3,
    "max_wind_speed": 15.9,
    "precipitation": "0.00I",
    "GHI_w/m2": 223
    }

Now, insert this document into our collection

In [4]:
result = climates.insert_one(newClimate)
print(result.inserted_id)

60af1c449343690fa94506b3


Let us look at the document that we have just inserted 

In [5]:
from pprint import pprint

cursor = climates.find({})
for document in cursor: 
    pprint(document)

{'GHI_w/m2': 223,
 '_id': ObjectId('60af1c449343690fa94506b3'),
 'air_temperature_celcius': 28,
 'date': '27/12/2018',
 'max_wind_speed': 15.9,
 'precipitation': '0.00I',
 'relative_humidity': 58.3,
 'station': 948702,
 'windspeed_knots': 9.3}


There are four entries on `27th December 2018` in `hotspot_historic.csv`:

**Entry 1:**
- latitude: -37.966
- longitude: 145.051
- datetime: 2018-12-27T04:16:51
- confidence: 78
- date: 27/12/2018
- surface_temperature_celcius: 68

**Entry 2:**
- latitude: -35.541
- longitude: 143.311
- datetime: 2018-12-27T00:02:15
- confidence: 82
- date: 27/12/2018
- surface_temperature_celcius: 63

**Entry 3:**
- latitude: -35.554
- longitude: 143.307
- datetime: 2018-12-27T00:02:15
- confidence: 67
- date: 27/12/2018
- surface_temperature_celcius: 53

**Entry 4:**
- latitude: -35.543
- longitude: 143.316
- datetime: 2018-12-27T00:02:14
- confidence: 86
- date: 27/12/2018
- surface_temperature_celcius: 67

Let us create the new documents for these entries so that we can add them to our database. 

In [6]:
#Create/get collection for fires
fires = db.fires
#Clear collection 
fires.drop()

In [7]:
newFires = [
    {
    
        "latitude": -37.966,
        "longitude": 145.051,
        "datetime": "2018-12-27T04:16:51",
        "confidence": 78,
        "date": "27/12/2018",
        "surface_temperature_celcius": 68

    },
    {
    
        "latitude": -35.541,
        "longitude": 143.311,
        "datetime": "2018-12-27T00:02:15",
        "confidence": 82,
        "date": "27/12/2018",
        "surface_temperature_celcius": 63

    },
    {
    
        "latitude": -35.554,
        "longitude": 143.307,
        "datetime": "2018-12-27T00:02:15",
        "confidence": 67,
        "date": "27/12/2018",
        "surface_temperature_celcius": 53

    },
    {
    
    "latitude": -35.543,
    "longitude": 143.316,
    "datetime": "2018-12-27T00:02:14",
    "confidence": 86,
    "date": "27/12/2018",
    "surface_temperature_celcius": 67

    }
    
]
result = fires.insert_many(newFires)

result.inserted_ids

[ObjectId('60af1c4b9343690fa94506b4'),
 ObjectId('60af1c4b9343690fa94506b5'),
 ObjectId('60af1c4b9343690fa94506b6'),
 ObjectId('60af1c4b9343690fa94506b7')]

Now look at the documents in fires collection

In [8]:
from pprint import pprint

cursor = fires.find({})
for document in cursor: 
    pprint(document)

{'_id': ObjectId('60af1c4b9343690fa94506b4'),
 'confidence': 78,
 'date': '27/12/2018',
 'datetime': '2018-12-27T04:16:51',
 'latitude': -37.966,
 'longitude': 145.051,
 'surface_temperature_celcius': 68}
{'_id': ObjectId('60af1c4b9343690fa94506b5'),
 'confidence': 82,
 'date': '27/12/2018',
 'datetime': '2018-12-27T00:02:15',
 'latitude': -35.541,
 'longitude': 143.311,
 'surface_temperature_celcius': 63}
{'_id': ObjectId('60af1c4b9343690fa94506b6'),
 'confidence': 67,
 'date': '27/12/2018',
 'datetime': '2018-12-27T00:02:15',
 'latitude': -35.554,
 'longitude': 143.307,
 'surface_temperature_celcius': 53}
{'_id': ObjectId('60af1c4b9343690fa94506b7'),
 'confidence': 86,
 'date': '27/12/2018',
 'datetime': '2018-12-27T00:02:14',
 'latitude': -35.543,
 'longitude': 143.316,
 'surface_temperature_celcius': 67}


Now, retrieve the unique ID's of the document in `fires` on the date that we want to reference in each corresponding `climates` entry and add them into an array

In [9]:
results = fires.find({"date":"27/12/2018"} , {"id_":1})

fires_id  = []
for result in results:
    fires_id.append(result["_id"])
    
print(fires_id)

[ObjectId('60af1c4b9343690fa94506b4'), ObjectId('60af1c4b9343690fa94506b5'), ObjectId('60af1c4b9343690fa94506b6'), ObjectId('60af1c4b9343690fa94506b7')]


We can now add the field `fires` to refer to the corresponding `fires` entries to each `climates` document.

In [10]:

result = climates.update_many({"date":"27/12/2018"},
    { 
    "$set": {
        "fires": fires_id} 
    })


Now we can check the `climates` collection once again to see the added field. 

In [11]:
cursor = climates.find({})
for document in cursor: 
    pprint(document)

{'GHI_w/m2': 223,
 '_id': ObjectId('60af1c449343690fa94506b3'),
 'air_temperature_celcius': 28,
 'date': '27/12/2018',
 'fires': [ObjectId('60af1c4b9343690fa94506b4'),
           ObjectId('60af1c4b9343690fa94506b5'),
           ObjectId('60af1c4b9343690fa94506b6'),
           ObjectId('60af1c4b9343690fa94506b7')],
 'max_wind_speed': 15.9,
 'precipitation': '0.00I',
 'relative_humidity': 58.3,
 'station': 948702,
 'windspeed_knots': 9.3}


We have now successfully stored the `fires` references inside the `climates` documents!

To understand why I have chosen this data model design, we first have to look at the two ways we can model `one-to-many` relationships:
- Embedding Documents
- Referencing Documents

**1. Embedding Documents**

If we were to choose this data model design, then we have to embed the  `fires` documents inside each corresponding `climates` document. This is not ideal because we can run out of memory.

**2. Referencing Documents**

Thus, to avoid this risk of running out of memory, we use references which allows us to keep our `climates` data in a separate collection from the `fires` collection. Since we now have two separate colections for `climates` and `fires`, not only do we have more memory for each collection but we can also access each collection independently. Now, we have two ways of referencing the data:
 
- Having `one` refence to the corresponding `climates` document in each `fires`document.
    * This is not ideal because there may be days where no fires were reported whereas climates are recorded on a daily basis so we risk missing out some climates data.

- Keeping an `array` of references to the corresponding `fires` document in each `climates` document.
    * This is better because we just just access the corresponding `fires` documents (if any) through each `climates` document.
    
This explains the justification behind choosing this data model. We will now move on to Task 2. 

**Task 2. Querying MongoDB using PyMongo**

Now that we understand the data model, we will move on to creating the actual database for part A.

**Load climate data to our database**

In [12]:
import pymongo
from pymongo import MongoClient

# Connect on the default host and port
client = MongoClient () 
#Create/get database 
db = client.fit3182_assignment_db
#Create/get collection for climate data
climates = db.climate_historic
#clear collection
climates.drop()

In [13]:
#import necessary modules
import csv
import datetime as dt

#open csv file containing climate data (https://stackoverflow.com/questions/17262256/how-to-read-one-single-line-of-csv-data-in-python)

header = None
with open('climate_historic.csv', newline='') as f:
    reader = csv.reader(f)
    #get the header (first row)
    header = next(reader)  
    
    #iterate row 2 onwards
    for row in reader:
        #we want to use the same format for date as in the fires data
        current_date = row[1]
        #initialise string for new date
        new_date = ""

        #reformat date
        for i in range (len(current_date)-2):
            new_date += current_date[i]

        new_date += "20"

        new_date += current_date[len(current_date)-2]
        new_date += current_date[len(current_date)-1]

        new_date = dt.datetime.strptime(new_date, "%d/%m/%Y").strftime("%d/%m/%Y")
        
        #create new document
        newClimate = {
                        #station
                        header[0] : int(row[0]),    
                        #date
                        header[1]: new_date,   
                        #air_temperature_celcius
                        header[2]: int(row[2]),
                        #relative_humidity
                        header[3]: float(row[3]),
                        #windspeed_knots
                        header[4]: float(row[4]),
                        #max_wind_speed
                        header[5]: float(row[5]),
                        #precipitation
                        header[6]: str(row[6]),
                        #GHI_w/m2
                        header[7]: int(row[7])
                      } 
        
        #insert document to climates colllection
        result = climates.insert_one(newClimate)


Now let us look at the inserted documents in `climates`

In [14]:
from pprint import pprint

cursor = climates.find({})
for document in cursor: 
    pprint(document)

{'GHI_w/m2': 154,
 '_id': ObjectId('60af1c5e9343690fa94506b9'),
 'air_temperature_celcius': 19,
 'date': '31/12/2017',
 'max_wind_speed': 11.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 56.8,
 'station': 948700,
 'windspeed_knots': 7.9}
{'GHI_w/m2': 128,
 '_id': ObjectId('60af1c5e9343690fa94506ba'),
 'air_temperature_celcius': 15,
 'date': '02/01/2018',
 'max_wind_speed': 13.0,
 'precipitation ': ' 0.02G',
 'relative_humidity': 50.7,
 'station': 948700,
 'windspeed_knots': 9.2}
{'GHI_w/m2': 133,
 '_id': ObjectId('60af1c5e9343690fa94506bb'),
 'air_temperature_celcius': 16,
 'date': '03/01/2018',
 'max_wind_speed': 15.0,
 'precipitation ': ' 0.00G',
 'relative_humidity': 53.6,
 'station': 948700,
 'windspeed_knots': 8.1}
{'GHI_w/m2': 186,
 '_id': ObjectId('60af1c5e9343690fa94506bc'),
 'air_temperature_celcius': 24,
 'date': '04/01/2018',
 'max_wind_speed': 14.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 61.6,
 'station': 948700,
 'windspeed_knots': 7.7}
{'GHI_w/m2': 185

 'air_temperature_celcius': 14,
 'date': '25/04/2018',
 'max_wind_speed': 18.1,
 'precipitation ': ' 0.98G',
 'relative_humidity': 52.8,
 'station': 948701,
 'windspeed_knots': 9.3}
{'GHI_w/m2': 102,
 '_id': ObjectId('60af1c5e9343690fa945072c'),
 'air_temperature_celcius': 11,
 'date': '26/04/2018',
 'max_wind_speed': 20.0,
 'precipitation ': ' 0.24G',
 'relative_humidity': 40.8,
 'station': 948701,
 'windspeed_knots': 12.2}
{'GHI_w/m2': 104,
 '_id': ObjectId('60af1c5e9343690fa945072d'),
 'air_temperature_celcius': 12,
 'date': '27/04/2018',
 'max_wind_speed': 11.1,
 'precipitation ': ' 0.03G',
 'relative_humidity': 48.2,
 'station': 948701,
 'windspeed_knots': 7.3}
{'GHI_w/m2': 132,
 '_id': ObjectId('60af1c5e9343690fa945072e'),
 'air_temperature_celcius': 16,
 'date': '28/04/2018',
 'max_wind_speed': 9.9,
 'precipitation ': ' 0.00G',
 'relative_humidity': 54.8,
 'station': 948701,
 'windspeed_knots': 8.3}
{'GHI_w/m2': 127,
 '_id': ObjectId('60af1c5e9343690fa945072f'),
 'air_temperatur

 'windspeed_knots': 3.1}
{'GHI_w/m2': 65,
 '_id': ObjectId('60af1c5e9343690fa945078d'),
 'air_temperature_celcius': 7,
 'date': '01/08/2018',
 'max_wind_speed': 7.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 40.7,
 'station': 948701,
 'windspeed_knots': 3.7}
{'GHI_w/m2': 47,
 '_id': ObjectId('60af1c5e9343690fa945078e'),
 'air_temperature_celcius': 5,
 'date': '02/08/2018',
 'max_wind_speed': 5.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 38.6,
 'station': 948701,
 'windspeed_knots': 1.8}
{'GHI_w/m2': 72,
 '_id': ObjectId('60af1c5e9343690fa945078f'),
 'air_temperature_celcius': 8,
 'date': '03/08/2018',
 'max_wind_speed': 8.0,
 'precipitation ': ' 0.39G',
 'relative_humidity': 44.6,
 'station': 948701,
 'windspeed_knots': 3.9}
{'GHI_w/m2': 92,
 '_id': ObjectId('60af1c5e9343690fa9450790'),
 'air_temperature_celcius': 10,
 'date': '04/08/2018',
 'max_wind_speed': 20.0,
 'precipitation ': ' 0.02G',
 'relative_humidity': 41.0,
 'station': 948702,
 'windspeed_knots': 14.1}


 'station': 948702,
 'windspeed_knots': 7.7}
{'GHI_w/m2': 118,
 '_id': ObjectId('60af1c5e9343690fa94507ed'),
 'air_temperature_celcius': 13,
 'date': '05/11/2018',
 'max_wind_speed': 14.0,
 'precipitation ': ' 0.00G',
 'relative_humidity': 43.3,
 'station': 948702,
 'windspeed_knots': 9.4}
{'GHI_w/m2': 117,
 '_id': ObjectId('60af1c5e9343690fa94507ee'),
 'air_temperature_celcius': 13,
 'date': '06/11/2018',
 'max_wind_speed': 19.0,
 'precipitation ': ' 0.02G',
 'relative_humidity': 44.1,
 'station': 948702,
 'windspeed_knots': 12.9}
{'GHI_w/m2': 109,
 '_id': ObjectId('60af1c5e9343690fa94507ef'),
 'air_temperature_celcius': 12,
 'date': '07/11/2018',
 'max_wind_speed': 15.9,
 'precipitation ': ' 0.01G',
 'relative_humidity': 42.7,
 'station': 948702,
 'windspeed_knots': 10.0}
{'GHI_w/m2': 106,
 '_id': ObjectId('60af1c5e9343690fa94507f0'),
 'air_temperature_celcius': 12,
 'date': '08/11/2018',
 'max_wind_speed': 14.0,
 'precipitation ': ' 0.00G',
 'relative_humidity': 46.6,
 'station': 94

**Load fire data to our database**


In [15]:
#create collection for fire data
fires = db.fire_historic
#clear collection
fires.drop()

In [16]:
#import necessary modules
import csv
import datetime as dt

#open csv file containing climate data (https://stackoverflow.com/questions/17262256/how-to-read-one-single-line-of-csv-data-in-python)

header = None
with open('hotspot_historic.csv', newline='') as f:
    reader = csv.reader(f)
    #get the header (first row)
    header = next(reader)  
    
    #iterate row 2 onwards
    for row in reader:
        
        #we want the format of the date to be the same for both collections
        current_date = row[4]
        new_date = dt.datetime.strptime(current_date, "%d/%m/%Y").strftime("%d/%m/%Y")
            
        #create new document
        newFire = {
                        #latitude
                        header[0] : float(row[0]),    
                        #longitude
                        header[1]: float(row[1]),   
                        #datetime
                        header[2]: str(row[2]),
                        #confidence
                        header[3]: int(row[3]),
                        #date
                        header[4]: new_date,
                        #surface_temperature_celcius
                        header[5]: int(row[5]),
                        
                      } 
        
        #insert document to fires colllection
        result = fires.insert_one(newFire)
        


Now let us look at the inserted documents in `fires`

In [17]:
cursor = fires.find({})
for document in cursor: 
    pprint(document)

{'_id': ObjectId('60af1c679343690fa9450827'),
 'confidence': 78,
 'date': '27/12/2018',
 'datetime': '2018-12-27T04:16:51',
 'latitude': -37.966,
 'longitude': 145.051,
 'surface_temperature_celcius': 68}
{'_id': ObjectId('60af1c679343690fa9450828'),
 'confidence': 82,
 'date': '27/12/2018',
 'datetime': '2018-12-27T00:02:15',
 'latitude': -35.541,
 'longitude': 143.311,
 'surface_temperature_celcius': 63}
{'_id': ObjectId('60af1c679343690fa9450829'),
 'confidence': 67,
 'date': '27/12/2018',
 'datetime': '2018-12-27T00:02:15',
 'latitude': -35.554,
 'longitude': 143.307,
 'surface_temperature_celcius': 53}
{'_id': ObjectId('60af1c679343690fa945082a'),
 'confidence': 86,
 'date': '27/12/2018',
 'datetime': '2018-12-27T00:02:14',
 'latitude': -35.543,
 'longitude': 143.316,
 'surface_temperature_celcius': 67}
{'_id': ObjectId('60af1c679343690fa945082b'),
 'confidence': 80,
 'date': '25/12/2018',
 'datetime': '2018-12-25T04:29:08',
 'latitude': -37.708,
 'longitude': 145.1,
 'surface_tem

 'date': '17/10/2018',
 'datetime': '2018-10-17T04:10:38',
 'latitude': -37.587,
 'longitude': 142.479,
 'surface_temperature_celcius': 41}
{'_id': ObjectId('60af1c689343690fa94508bd'),
 'confidence': 82,
 'date': '17/10/2018',
 'datetime': '2018-10-17T04:10:32',
 'latitude': -37.805,
 'longitude': 144.15,
 'surface_temperature_celcius': 55}
{'_id': ObjectId('60af1c689343690fa94508be'),
 'confidence': 62,
 'date': '16/10/2018',
 'datetime': '2018-10-16T23:56:43',
 'latitude': -37.288,
 'longitude': 144.39,
 'surface_temperature_celcius': 36}
{'_id': ObjectId('60af1c689343690fa94508bf'),
 'confidence': 100,
 'date': '15/10/2018',
 'datetime': '2018-10-15T04:23:01',
 'latitude': -37.294,
 'longitude': 141.232,
 'surface_temperature_celcius': 87}
{'_id': ObjectId('60af1c689343690fa94508c0'),
 'confidence': 99,
 'date': '15/10/2018',
 'datetime': '2018-10-15T04:23:01',
 'latitude': -37.293,
 'longitude': 141.245,
 'surface_temperature_celcius': 85}
{'_id': ObjectId('60af1c689343690fa94508c

 'longitude': 142.204,
 'surface_temperature_celcius': 58}
{'_id': ObjectId('60af1c689343690fa9450937'),
 'confidence': 86,
 'date': '10/09/2018',
 'datetime': '2018-09-10T03:50:30',
 'latitude': -34.2803,
 'longitude': 142.1964,
 'surface_temperature_celcius': 60}
{'_id': ObjectId('60af1c689343690fa9450938'),
 'confidence': 81,
 'date': '10/09/2018',
 'datetime': '2018-09-10T03:50:10',
 'latitude': -34.2782,
 'longitude': 142.2253,
 'surface_temperature_celcius': 54}
{'_id': ObjectId('60af1c689343690fa9450939'),
 'confidence': 56,
 'date': '14/08/2018',
 'datetime': '2018-08-14T04:15:40',
 'latitude': -37.4421,
 'longitude': 148.259,
 'surface_temperature_celcius': 39}
{'_id': ObjectId('60af1c689343690fa945093a'),
 'confidence': 61,
 'date': '14/08/2018',
 'datetime': '2018-08-14T04:11:00',
 'latitude': -38.0173,
 'longitude': 144.0571,
 'surface_temperature_celcius': 41}
{'_id': ObjectId('60af1c689343690fa945093b'),
 'confidence': 67,
 'date': '14/08/2018',
 'datetime': '2018-08-14T0

 'surface_temperature_celcius': 59}
{'_id': ObjectId('60af1c689343690fa94509cd'),
 'confidence': 74,
 'date': '22/05/2018',
 'datetime': '2018-05-22T00:13:10',
 'latitude': -38.4412,
 'longitude': 146.3188,
 'surface_temperature_celcius': 48}
{'_id': ObjectId('60af1c689343690fa94509ce'),
 'confidence': 61,
 'date': '22/05/2018',
 'datetime': '2018-05-22T00:13:00',
 'latitude': -38.4322,
 'longitude': 146.3208,
 'surface_temperature_celcius': 42}
{'_id': ObjectId('60af1c689343690fa94509cf'),
 'confidence': 66,
 'date': '22/05/2018',
 'datetime': '2018-05-22T00:13:00',
 'latitude': -37.9077,
 'longitude': 141.1101,
 'surface_temperature_celcius': 43}
{'_id': ObjectId('60af1c689343690fa94509d0'),
 'confidence': 100,
 'date': '22/05/2018',
 'datetime': '2018-05-22T00:13:00',
 'latitude': -37.9047,
 'longitude': 141.0945,
 'surface_temperature_celcius': 90}
{'_id': ObjectId('60af1c689343690fa94509d1'),
 'confidence': 89,
 'date': '22/05/2018',
 'datetime': '2018-05-22T00:13:00',
 'latitude'

 'latitude': -36.591,
 'longitude': 141.7194,
 'surface_temperature_celcius': 40}
{'_id': ObjectId('60af1c689343690fa9450a74'),
 'confidence': 68,
 'date': '13/05/2018',
 'datetime': '2018-05-13T04:38:40',
 'latitude': -36.8377,
 'longitude': 142.642,
 'surface_temperature_celcius': 44}
{'_id': ObjectId('60af1c689343690fa9450a75'),
 'confidence': 100,
 'date': '13/05/2018',
 'datetime': '2018-05-13T04:38:40',
 'latitude': -36.1506,
 'longitude': 143.5671,
 'surface_temperature_celcius': 95}
{'_id': ObjectId('60af1c689343690fa9450a76'),
 'confidence': 86,
 'date': '13/05/2018',
 'datetime': '2018-05-13T04:38:40',
 'latitude': -35.7108,
 'longitude': 143.7836,
 'surface_temperature_celcius': 60}
{'_id': ObjectId('60af1c689343690fa9450a77'),
 'confidence': 63,
 'date': '13/05/2018',
 'datetime': '2018-05-13T04:38:40',
 'latitude': -36.0518,
 'longitude': 141.6846,
 'surface_temperature_celcius': 42}
{'_id': ObjectId('60af1c689343690fa9450a78'),
 'confidence': 85,
 'date': '13/05/2018',
 '

...

 'surface_temperature_celcius': 81}
{'_id': ObjectId('60af1c689343690fa9450f66'),
 'confidence': 86,
 'date': '13/04/2018',
 'datetime': '2018-04-13T04:26:30',
 'latitude': -36.4631,
 'longitude': 144.7654,
 'surface_temperature_celcius': 61}
{'_id': ObjectId('60af1c689343690fa9450f67'),
 'confidence': 91,
 'date': '13/04/2018',
 'datetime': '2018-04-13T04:26:30',
 'latitude': -36.4612,
 'longitude': 144.7775,
 'surface_temperature_celcius': 80}
{'_id': ObjectId('60af1c689343690fa9450f68'),
 'confidence': 68,
 'date': '13/04/2018',
 'datetime': '2018-04-13T04:26:30',
 'latitude': -36.3275,
 'longitude': 145.5876,
 'surface_temperature_celcius': 44}
{'_id': ObjectId('60af1c689343690fa9450f69'),
 'confidence': 81,
 'date': '13/04/2018',
 'datetime': '2018-04-13T04:26:30',
 'latitude': -36.1833,
 'longitude': 143.542,
 'surface_temperature_celcius': 55}
{'_id': ObjectId('60af1c689343690fa9450f6a'),
 'confidence': 100,
 'date': '13/04/2018',
 'datetime': '2018-04-13T04:26:30',
 'latitude':

 'latitude': -36.6507,
 'longitude': 142.4998,
 'surface_temperature_celcius': 61}
{'_id': ObjectId('60af1c689343690fa945100d'),
 'confidence': 92,
 'date': '12/04/2018',
 'datetime': '2018-04-12T03:47:50',
 'latitude': -37.4033,
 'longitude': 143.6884,
 'surface_temperature_celcius': 70}
{'_id': ObjectId('60af1c689343690fa945100e'),
 'confidence': 84,
 'date': '12/04/2018',
 'datetime': '2018-04-12T03:47:50',
 'latitude': -36.8292,
 'longitude': 142.1692,
 'surface_temperature_celcius': 58}
{'_id': ObjectId('60af1c689343690fa945100f'),
 'confidence': 84,
 'date': '12/04/2018',
 'datetime': '2018-04-12T03:47:30',
 'latitude': -36.1456,
 'longitude': 141.9066,
 'surface_temperature_celcius': 58}
{'_id': ObjectId('60af1c689343690fa9451010'),
 'confidence': 70,
 'date': '12/04/2018',
 'datetime': '2018-04-12T03:47:30',
 'latitude': -36.6348,
 'longitude': 142.4985,
 'surface_temperature_celcius': 45}
{'_id': ObjectId('60af1c689343690fa9451011'),
 'confidence': 100,
 'date': '12/04/2018',


 'confidence': 85,
 'date': '06/04/2018',
 'datetime': '2018-04-06T04:22:20',
 'latitude': -37.8078,
 'longitude': 143.3482,
 'surface_temperature_celcius': 60}
{'_id': ObjectId('60af1c689343690fa94510b4'),
 'confidence': 72,
 'date': '06/04/2018',
 'datetime': '2018-04-06T04:22:20',
 'latitude': -37.8297,
 'longitude': 142.4083,
 'surface_temperature_celcius': 48}
{'_id': ObjectId('60af1c689343690fa94510b5'),
 'confidence': 100,
 'date': '06/04/2018',
 'datetime': '2018-04-06T04:22:20',
 'latitude': -36.3168,
 'longitude': 145.7285,
 'surface_temperature_celcius': 103}
{'_id': ObjectId('60af1c689343690fa94510b6'),
 'confidence': 51,
 'date': '06/04/2018',
 'datetime': '2018-04-06T04:22:20',
 'latitude': -37.8177,
 'longitude': 142.9168,
 'surface_temperature_celcius': 50}
{'_id': ObjectId('60af1c689343690fa94510b7'),
 'confidence': 91,
 'date': '06/04/2018',
 'datetime': '2018-04-06T04:22:20',
 'latitude': -36.031,
 'longitude': 145.6824,
 'surface_temperature_celcius': 69}
{'_id': Ob

 'surface_temperature_celcius': 52}
{'_id': ObjectId('60af1c689343690fa945115a'),
 'confidence': 54,
 'date': '04/04/2018',
 'datetime': '2018-04-04T04:34:20',
 'latitude': -38.4879,
 'longitude': 143.9971,
 'surface_temperature_celcius': 40}
{'_id': ObjectId('60af1c689343690fa945115b'),
 'confidence': 96,
 'date': '04/04/2018',
 'datetime': '2018-04-04T04:34:20',
 'latitude': -36.1955,
 'longitude': 145.914,
 'surface_temperature_celcius': 78}
{'_id': ObjectId('60af1c689343690fa945115c'),
 'confidence': 85,
 'date': '04/04/2018',
 'datetime': '2018-04-04T04:34:20',
 'latitude': -35.4533,
 'longitude': 143.2029,
 'surface_temperature_celcius': 66}
{'_id': ObjectId('60af1c689343690fa945115d'),
 'confidence': 84,
 'date': '04/04/2018',
 'datetime': '2018-04-04T04:34:00',
 'latitude': -35.4993,
 'longitude': 143.3105,
 'surface_temperature_celcius': 57}
{'_id': ObjectId('60af1c689343690fa945115e'),
 'confidence': 92,
 'date': '04/04/2018',
 'datetime': '2018-04-04T04:33:50',
 'latitude': 

 'datetime': '2018-03-28T04:33:20',
 'latitude': -36.4737,
 'longitude': 142.2474,
 'surface_temperature_celcius': 55}
{'_id': ObjectId('60af1c689343690fa94511fd'),
 'confidence': 93,
 'date': '28/03/2018',
 'datetime': '2018-03-28T04:32:20',
 'latitude': -36.8099,
 'longitude': 142.728,
 'surface_temperature_celcius': 73}
{'_id': ObjectId('60af1c689343690fa94511fe'),
 'confidence': 76,
 'date': '28/03/2018',
 'datetime': '2018-03-28T04:31:50',
 'latitude': -36.8202,
 'longitude': 141.9362,
 'surface_temperature_celcius': 50}
{'_id': ObjectId('60af1c689343690fa94511ff'),
 'confidence': 70,
 'date': '28/03/2018',
 'datetime': '2018-03-28T04:31:50',
 'latitude': -36.9576,
 'longitude': 141.5387,
 'surface_temperature_celcius': 45}
{'_id': ObjectId('60af1c689343690fa9451200'),
 'confidence': 83,
 'date': '28/03/2018',
 'datetime': '2018-03-28T04:30:50',
 'latitude': -36.811,
 'longitude': 142.2655,
 'surface_temperature_celcius': 56}
{'_id': ObjectId('60af1c689343690fa9451201'),
 'confide

We can now move on to adding the references in `climates` documents

In [18]:
cursor = climates.find({})
for document in cursor: 
    #get the date
    date = document["date"]
    
    #find the document in fires with corresponding date
    fires_result = fires.find({"date": date})
    #array to store the fire documents' ID's
    fires_id_array = []
    for fire_doc in fires_result:
        #get the id
        fire_id = fire_doc["_id"]
        
        #append to array
        fires_id_array.append(fire_id)
            
        
    #add fire id's references in climate document 
    climate_id = document["_id"]
    
    
    result = climates.update_one({"_id": climate_id},
    { 
    "$set": {
        "fires": fires_id_array} 
    })

Check the documents in `climates` again

In [19]:
cursor = climates.find({})
for document in cursor: 
    pprint(document)

{'GHI_w/m2': 154,
 '_id': ObjectId('60af1c5e9343690fa94506b9'),
 'air_temperature_celcius': 19,
 'date': '31/12/2017',
 'fires': [],
 'max_wind_speed': 11.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 56.8,
 'station': 948700,
 'windspeed_knots': 7.9}
{'GHI_w/m2': 128,
 '_id': ObjectId('60af1c5e9343690fa94506ba'),
 'air_temperature_celcius': 15,
 'date': '02/01/2018',
 'fires': [],
 'max_wind_speed': 13.0,
 'precipitation ': ' 0.02G',
 'relative_humidity': 50.7,
 'station': 948700,
 'windspeed_knots': 9.2}
{'GHI_w/m2': 133,
 '_id': ObjectId('60af1c5e9343690fa94506bb'),
 'air_temperature_celcius': 16,
 'date': '03/01/2018',
 'fires': [],
 'max_wind_speed': 15.0,
 'precipitation ': ' 0.00G',
 'relative_humidity': 53.6,
 'station': 948700,
 'windspeed_knots': 8.1}
{'GHI_w/m2': 186,
 '_id': ObjectId('60af1c5e9343690fa94506bc'),
 'air_temperature_celcius': 24,
 'date': '04/01/2018',
 'fires': [],
 'max_wind_speed': 14.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 61.6,
 'sta

           ObjectId('60af1c689343690fa945116b'),
           ObjectId('60af1c689343690fa945116c'),
           ObjectId('60af1c689343690fa945116d'),
           ObjectId('60af1c689343690fa945116e'),
           ObjectId('60af1c689343690fa945116f'),
           ObjectId('60af1c689343690fa9451170'),
           ObjectId('60af1c689343690fa9451171'),
           ObjectId('60af1c689343690fa9451172'),
           ObjectId('60af1c689343690fa9451173'),
           ObjectId('60af1c689343690fa9451174'),
           ObjectId('60af1c689343690fa9451175'),
           ObjectId('60af1c689343690fa9451176'),
           ObjectId('60af1c689343690fa9451177'),
           ObjectId('60af1c689343690fa9451178'),
           ObjectId('60af1c689343690fa9451179'),
           ObjectId('60af1c689343690fa945117a'),
           ObjectId('60af1c689343690fa945117b'),
           ObjectId('60af1c689343690fa945117c'),
           ObjectId('60af1c689343690fa945117d'),
           ObjectId('60af1c689343690fa945117e'),
           ObjectId(

           ObjectId('60af1c689343690fa9450bca'),
           ObjectId('60af1c689343690fa9450bcb'),
           ObjectId('60af1c689343690fa9450bcc'),
           ObjectId('60af1c689343690fa9450bcd'),
           ObjectId('60af1c689343690fa9450bce'),
           ObjectId('60af1c689343690fa9450bcf'),
           ObjectId('60af1c689343690fa9450bd0'),
           ObjectId('60af1c689343690fa9450bd1'),
           ObjectId('60af1c689343690fa9450bd2'),
           ObjectId('60af1c689343690fa9450bd3'),
           ObjectId('60af1c689343690fa9450bd4'),
           ObjectId('60af1c689343690fa9450bd5'),
           ObjectId('60af1c689343690fa9450bd6'),
           ObjectId('60af1c689343690fa9450bd7'),
           ObjectId('60af1c689343690fa9450bd8'),
           ObjectId('60af1c689343690fa9450bd9'),
           ObjectId('60af1c689343690fa9450bda'),
           ObjectId('60af1c689343690fa9450bdb'),
           ObjectId('60af1c689343690fa9450bdc'),
           ObjectId('60af1c689343690fa9450bdd'),
           ObjectId(

 'date': '10/07/2018',
 'fires': [],
 'max_wind_speed': 11.1,
 'precipitation ': ' 0.12G',
 'relative_humidity': 44.4,
 'station': 948701,
 'windspeed_knots': 5.6}
{'GHI_w/m2': 56,
 '_id': ObjectId('60af1c5e9343690fa9450778'),
 'air_temperature_celcius': 6,
 'date': '11/07/2018',
 'fires': [],
 'max_wind_speed': 5.1,
 'precipitation ': ' 0.01G',
 'relative_humidity': 39.0,
 'station': 948701,
 'windspeed_knots': 1.8}
{'GHI_w/m2': 84,
 '_id': ObjectId('60af1c5e9343690fa9450779'),
 'air_temperature_celcius': 9,
 'date': '12/07/2018',
 'fires': [],
 'max_wind_speed': 8.9,
 'precipitation ': ' 0.00G',
 'relative_humidity': 39.0,
 'station': 948701,
 'windspeed_knots': 5.6}
{'GHI_w/m2': 111,
 '_id': ObjectId('60af1c5e9343690fa945077a'),
 'air_temperature_celcius': 12,
 'date': '13/07/2018',
 'fires': [],
 'max_wind_speed': 21.0,
 'precipitation ': ' 0.12G',
 'relative_humidity': 40.7,
 'station': 948701,
 'windspeed_knots': 16.7}
{'GHI_w/m2': 84,
 '_id': ObjectId('60af1c5e9343690fa945077b')

 'date': '18/10/2018',
 'fires': [ObjectId('60af1c689343690fa94508b3'),
           ObjectId('60af1c689343690fa94508b4'),
           ObjectId('60af1c689343690fa94508b5'),
           ObjectId('60af1c689343690fa94508b6'),
           ObjectId('60af1c689343690fa94508b7'),
           ObjectId('60af1c689343690fa94508b8')],
 'max_wind_speed': 16.9,
 'precipitation ': ' 0.00I',
 'relative_humidity': 52.2,
 'station': 948702,
 'windspeed_knots': 13.7}
{'GHI_w/m2': 128,
 '_id': ObjectId('60af1c5e9343690fa94507dc'),
 'air_temperature_celcius': 15,
 'date': '19/10/2018',
 'fires': [],
 'max_wind_speed': 15.9,
 'precipitation ': ' 0.16G',
 'relative_humidity': 51.0,
 'station': 948702,
 'windspeed_knots': 9.6}
{'GHI_w/m2': 119,
 '_id': ObjectId('60af1c5e9343690fa94507dd'),
 'air_temperature_celcius': 13,
 'date': '20/10/2018',
 'fires': [ObjectId('60af1c689343690fa94508b0'),
           ObjectId('60af1c689343690fa94508b1'),
           ObjectId('60af1c689343690fa94508b2')],
 'max_wind_speed': 16.9,
 '

Now that we have loaded the data into the new database, we can perform the queries.

**a. Find climate data on 12th December 2018.**

In [20]:
results = climates.find({"date": "12/12/2018"})
for result in results: 
    pprint(result)

{'GHI_w/m2': 156,
 '_id': ObjectId('60af1c5e9343690fa9450812'),
 'air_temperature_celcius': 19,
 'date': '12/12/2018',
 'fires': [ObjectId('60af1c679343690fa9450843')],
 'max_wind_speed': 12.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 55.3,
 'station': 948702,
 'windspeed_knots': 6.2}


**b. Find the latitude, longitude, surface temperature (°C), and confidence when the surface temperature (°C) was between 65 °C and 100 °C.**


In [21]:
results = fires.find(
    {
        "$and":[
            {
                "surface_temperature_celcius":{"$gte":65}
            }, 
            {
                "surface_temperature_celcius":{"$lte":100}
            }
        ]
    },
    {
        "latitude": 1,
        "longitude": 1,
        "surface_temperature_celcius": 1,
        "confidence": 1
    }
)
for result in results:
    pprint(result)

{'_id': ObjectId('60af1c679343690fa9450827'),
 'confidence': 78,
 'latitude': -37.966,
 'longitude': 145.051,
 'surface_temperature_celcius': 68}
{'_id': ObjectId('60af1c679343690fa945082a'),
 'confidence': 86,
 'latitude': -35.543,
 'longitude': 143.316,
 'surface_temperature_celcius': 67}
{'_id': ObjectId('60af1c679343690fa9450831'),
 'confidence': 93,
 'latitude': -37.875,
 'longitude': 142.51,
 'surface_temperature_celcius': 73}
{'_id': ObjectId('60af1c679343690fa9450833'),
 'confidence': 95,
 'latitude': -37.613,
 'longitude': 149.305,
 'surface_temperature_celcius': 75}
{'_id': ObjectId('60af1c679343690fa9450835'),
 'confidence': 90,
 'latitude': -37.624,
 'longitude': 149.314,
 'surface_temperature_celcius': 66}
{'_id': ObjectId('60af1c679343690fa9450838'),
 'confidence': 93,
 'latitude': -38.057,
 'longitude': 144.211,
 'surface_temperature_celcius': 73}
{'_id': ObjectId('60af1c679343690fa9450841'),
 'confidence': 92,
 'latitude': -37.95,
 'longitude': 142.366,
 'surface_temper

 'latitude': -37.7756,
 'longitude': 143.1388,
 'surface_temperature_celcius': 67}
{'_id': ObjectId('60af1c689343690fa9450d78'),
 'confidence': 91,
 'latitude': -37.7171,
 'longitude': 142.9505,
 'surface_temperature_celcius': 68}
{'_id': ObjectId('60af1c689343690fa9450d79'),
 'confidence': 94,
 'latitude': -37.7236,
 'longitude': 142.947,
 'surface_temperature_celcius': 73}
{'_id': ObjectId('60af1c689343690fa9450d7b'),
 'confidence': 100,
 'latitude': -37.7002,
 'longitude': 142.9562,
 'surface_temperature_celcius': 89}
{'_id': ObjectId('60af1c689343690fa9450d85'),
 'confidence': 92,
 'latitude': -37.6498,
 'longitude': 142.5997,
 'surface_temperature_celcius': 70}
{'_id': ObjectId('60af1c689343690fa9450d89'),
 'confidence': 100,
 'latitude': -37.5728,
 'longitude': 142.6348,
 'surface_temperature_celcius': 98}
{'_id': ObjectId('60af1c689343690fa9450d8c'),
 'confidence': 100,
 'latitude': -37.4352,
 'longitude': 143.1444,
 'surface_temperature_celcius': 96}
{'_id': ObjectId('60af1c689

{'_id': ObjectId('60af1c689343690fa9451121'),
 'confidence': 89,
 'latitude': -37.4471,
 'longitude': 147.6331,
 'surface_temperature_celcius': 65}
{'_id': ObjectId('60af1c689343690fa9451124'),
 'confidence': 90,
 'latitude': -37.7835,
 'longitude': 148.4138,
 'surface_temperature_celcius': 66}
{'_id': ObjectId('60af1c689343690fa9451127'),
 'confidence': 89,
 'latitude': -37.4449,
 'longitude': 147.6594,
 'surface_temperature_celcius': 65}
{'_id': ObjectId('60af1c689343690fa9451129'),
 'confidence': 95,
 'latitude': -37.4323,
 'longitude': 147.6583,
 'surface_temperature_celcius': 76}
{'_id': ObjectId('60af1c689343690fa945112a'),
 'confidence': 100,
 'latitude': -37.4343,
 'longitude': 147.6327,
 'surface_temperature_celcius': 92}
{'_id': ObjectId('60af1c689343690fa945113c'),
 'confidence': 90,
 'latitude': -35.4033,
 'longitude': 143.267,
 'surface_temperature_celcius': 67}
{'_id': ObjectId('60af1c689343690fa945113d'),
 'confidence': 64,
 'latitude': -36.6281,
 'longitude': 142.4349,


**c. Find date, surface temperature (°C), air temperature (°C), relative humidity and max wind speed on 15th and 16th of December 2018.**


In [22]:
results = climates.aggregate(
    [
        {
            "$match": { "$or": [{ "date": '15/12/2018' }, {"date": '16/12/2018' }] }
        },
        {
            "$lookup":{
                    "from": "fire_historic",
                    "localField": "fires",
                    "foreignField" : "_id",
                    "as": "fires"
            }
        },
        {
            "$project": {
                "date": 1, 
                "fires.surface_temperature_celcius":1, 
                "air_temperature_celcius":1, 
                "relative_humidity": 1,
                "max_wind_speed": 1, 
            }
        }
    ]
)

for document in results:
    pprint(document)

{'_id': ObjectId('60af1c5e9343690fa9450815'),
 'air_temperature_celcius': 18,
 'date': '15/12/2018',
 'fires': [{'surface_temperature_celcius': 42},
           {'surface_temperature_celcius': 36},
           {'surface_temperature_celcius': 38},
           {'surface_temperature_celcius': 40}],
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'_id': ObjectId('60af1c5e9343690fa9450816'),
 'air_temperature_celcius': 18,
 'date': '16/12/2018',
 'fires': [{'surface_temperature_celcius': 43},
           {'surface_temperature_celcius': 33},
           {'surface_temperature_celcius': 54},
           {'surface_temperature_celcius': 73},
           {'surface_temperature_celcius': 55},
           {'surface_temperature_celcius': 75},
           {'surface_temperature_celcius': 55},
           {'surface_temperature_celcius': 66},
           {'surface_temperature_celcius': 56},
           {'surface_temperature_celcius': 60},
           {'surface_temperature_celcius': 73},
           {'surface_tem

**d. Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.**

In [23]:
results = climates.aggregate(
    [
        
        {
            "$lookup":{
                    "from": "fire_historic",
                    "localField": "fires",
                    "foreignField" : "_id",
                    "as": "fires"
            }
        },
        {
             "$unwind": "$fires"
        },
        {
            "$match": { "$and": [{ "fires.confidence": { "$gte": 80 } }, {"fires.confidence": { "$lte": 100 } }] }
        },
        {
            "$project": {
                "fires.datetime": 1, 
                "air_temperature_celcius":1, 
                "fires.surface_temperature_celcius":1, 
                "fires.confidence": 1, 
            }
        }
    ]
)

for document in results:
    pprint(document)

{'_id': ObjectId('60af1c5e9343690fa94506f9'),
 'air_temperature_celcius': 20,
 'fires': {'confidence': 87,
           'datetime': '2018-03-06T05:06:30',
           'surface_temperature_celcius': 62}}
{'_id': ObjectId('60af1c5e9343690fa94506f9'),
 'air_temperature_celcius': 20,
 'fires': {'confidence': 85,
           'datetime': '2018-03-06T05:06:20',
           'surface_temperature_celcius': 59}}
{'_id': ObjectId('60af1c5e9343690fa94506fa'),
 'air_temperature_celcius': 19,
 'fires': {'confidence': 88,
           'datetime': '2018-03-07T04:16:10',
           'surface_temperature_celcius': 64}}
{'_id': ObjectId('60af1c5e9343690fa94506fc'),
 'air_temperature_celcius': 23,
 'fires': {'confidence': 86,
           'datetime': '2018-03-09T13:23:40',
           'surface_temperature_celcius': 41}}
{'_id': ObjectId('60af1c5e9343690fa94506fd'),
 'air_temperature_celcius': 19,
 'fires': {'confidence': 100,
           'datetime': '2018-03-10T04:48:40',
           'surface_temperature_celcius': 105}

           'surface_temperature_celcius': 40}}
{'_id': ObjectId('60af1c5e9343690fa9450717'),
 'air_temperature_celcius': 17,
 'fires': {'confidence': 100,
           'datetime': '2018-04-05T13:04:20',
           'surface_temperature_celcius': 49}}
{'_id': ObjectId('60af1c5e9343690fa9450717'),
 'air_temperature_celcius': 17,
 'fires': {'confidence': 100,
           'datetime': '2018-04-05T13:03:20',
           'surface_temperature_celcius': 55}}
{'_id': ObjectId('60af1c5e9343690fa9450717'),
 'air_temperature_celcius': 17,
 'fires': {'confidence': 98,
           'datetime': '2018-04-05T13:03:00',
           'surface_temperature_celcius': 45}}
{'_id': ObjectId('60af1c5e9343690fa9450717'),
 'air_temperature_celcius': 17,
 'fires': {'confidence': 100,
           'datetime': '2018-04-05T13:03:00',
           'surface_temperature_celcius': 55}}
{'_id': ObjectId('60af1c5e9343690fa9450717'),
 'air_temperature_celcius': 17,
 'fires': {'confidence': 81,
           'datetime': '2018-04-05T13:03:00

 'fires': {'confidence': 86,
           'datetime': '2018-04-13T04:36:50',
           'surface_temperature_celcius': 60}}
{'_id': ObjectId('60af1c5e9343690fa945071f'),
 'air_temperature_celcius': 16,
 'fires': {'confidence': 92,
           'datetime': '2018-04-13T04:36:30',
           'surface_temperature_celcius': 70}}
{'_id': ObjectId('60af1c5e9343690fa945071f'),
 'air_temperature_celcius': 16,
 'fires': {'confidence': 89,
           'datetime': '2018-04-13T04:36:20',
           'surface_temperature_celcius': 65}}
{'_id': ObjectId('60af1c5e9343690fa945071f'),
 'air_temperature_celcius': 16,
 'fires': {'confidence': 80,
           'datetime': '2018-04-13T04:36:20',
           'surface_temperature_celcius': 53}}
{'_id': ObjectId('60af1c5e9343690fa945071f'),
 'air_temperature_celcius': 16,
 'fires': {'confidence': 89,
           'datetime': '2018-04-13T04:35:40',
           'surface_temperature_celcius': 64}}
{'_id': ObjectId('60af1c5e9343690fa945071f'),
 'air_temperature_celcius': 16,


 'air_temperature_celcius': 12,
 'fires': {'confidence': 82,
           'datetime': '2018-04-15T04:16:00',
           'surface_temperature_celcius': 55}}
{'_id': ObjectId('60af1c5e9343690fa9450721'),
 'air_temperature_celcius': 12,
 'fires': {'confidence': 82,
           'datetime': '2018-04-15T04:15:40',
           'surface_temperature_celcius': 56}}
{'_id': ObjectId('60af1c5e9343690fa9450721'),
 'air_temperature_celcius': 12,
 'fires': {'confidence': 82,
           'datetime': '2018-04-15T04:15:10',
           'surface_temperature_celcius': 55}}
{'_id': ObjectId('60af1c5e9343690fa9450721'),
 'air_temperature_celcius': 12,
 'fires': {'confidence': 88,
           'datetime': '2018-04-15T04:15:10',
           'surface_temperature_celcius': 63}}
{'_id': ObjectId('60af1c5e9343690fa9450721'),
 'air_temperature_celcius': 12,
 'fires': {'confidence': 84,
           'datetime': '2018-04-15T04:15:10',
           'surface_temperature_celcius': 58}}
{'_id': ObjectId('60af1c5e9343690fa9450721'),


 'fires': {'confidence': 88,
           'datetime': '2018-04-19T13:15:10',
           'surface_temperature_celcius': 42}}
{'_id': ObjectId('60af1c5e9343690fa9450725'),
 'air_temperature_celcius': 22,
 'fires': {'confidence': 87,
           'datetime': '2018-04-19T03:53:20',
           'surface_temperature_celcius': 62}}
{'_id': ObjectId('60af1c5e9343690fa9450725'),
 'air_temperature_celcius': 22,
 'fires': {'confidence': 89,
           'datetime': '2018-04-19T03:52:10',
           'surface_temperature_celcius': 65}}
{'_id': ObjectId('60af1c5e9343690fa9450725'),
 'air_temperature_celcius': 22,
 'fires': {'confidence': 88,
           'datetime': '2018-04-19T03:52:10',
           'surface_temperature_celcius': 74}}
{'_id': ObjectId('60af1c5e9343690fa9450725'),
 'air_temperature_celcius': 22,
 'fires': {'confidence': 88,
           'datetime': '2018-04-19T03:52:10',
           'surface_temperature_celcius': 64}}
{'_id': ObjectId('60af1c5e9343690fa9450725'),
 'air_temperature_celcius': 22,


 'fires': {'confidence': 86,
           'datetime': '2018-05-10T04:08:30',
           'surface_temperature_celcius': 61}}
{'_id': ObjectId('60af1c5e9343690fa945073a'),
 'air_temperature_celcius': 10,
 'fires': {'confidence': 83,
           'datetime': '2018-05-10T04:08:30',
           'surface_temperature_celcius': 56}}
{'_id': ObjectId('60af1c5e9343690fa945073a'),
 'air_temperature_celcius': 10,
 'fires': {'confidence': 95,
           'datetime': '2018-05-10T04:08:10',
           'surface_temperature_celcius': 76}}
{'_id': ObjectId('60af1c5e9343690fa945073a'),
 'air_temperature_celcius': 10,
 'fires': {'confidence': 85,
           'datetime': '2018-05-10T04:08:10',
           'surface_temperature_celcius': 59}}
{'_id': ObjectId('60af1c5e9343690fa945073a'),
 'air_temperature_celcius': 10,
 'fires': {'confidence': 80,
           'datetime': '2018-05-10T04:08:10',
           'surface_temperature_celcius': 54}}
{'_id': ObjectId('60af1c5e9343690fa945073a'),
 'air_temperature_celcius': 10,


 'fires': {'confidence': 89,
           'datetime': '2018-09-20T04:29:03',
           'surface_temperature_celcius': 60}}
{'_id': ObjectId('60af1c5e9343690fa94507bf'),
 'air_temperature_celcius': 14,
 'fires': {'confidence': 100,
           'datetime': '2018-09-20T04:29:03',
           'surface_temperature_celcius': 112}}
{'_id': ObjectId('60af1c5e9343690fa94507bf'),
 'air_temperature_celcius': 14,
 'fires': {'confidence': 88,
           'datetime': '2018-09-20T04:28:45',
           'surface_temperature_celcius': 60}}
{'_id': ObjectId('60af1c5e9343690fa94507c2'),
 'air_temperature_celcius': 17,
 'fires': {'confidence': 86,
           'datetime': '2018-09-23T23:50:44',
           'surface_temperature_celcius': 56}}
{'_id': ObjectId('60af1c5e9343690fa94507c2'),
 'air_temperature_celcius': 17,
 'fires': {'confidence': 100,
           'datetime': '2018-09-23T12:47:12',
           'surface_temperature_celcius': 56}}
{'_id': ObjectId('60af1c5e9343690fa94507c2'),
 'air_temperature_celcius': 1

**e. Find the top 10 records with the highest surface temperature (°C).**

In [24]:
results = climates.aggregate(
    [

        {
            "$lookup":{
                    "from": "fire_historic",
                    "localField": "fires",
                    "foreignField" : "_id",
                    "as": "fires"
            }
        },
        {
             "$unwind": "$fires"
        },
        {
            "$sort": {"fires.surface_temperature_celcius": -1}
        },
        {
            "$limit": 10
        }
    ]
)

for document in results:
    pprint(document)

{'GHI_w/m2': 122,
 '_id': ObjectId('60af1c5e9343690fa9450724'),
 'air_temperature_celcius': 15,
 'date': '18/04/2018',
 'fires': {'_id': ObjectId('60af1c689343690fa9450cca'),
           'confidence': 100,
           'date': '18/04/2018',
           'datetime': '2018-04-18T04:52:00',
           'latitude': -38.1665,
           'longitude': 143.062,
           'surface_temperature_celcius': 124},
 'max_wind_speed': 9.9,
 'precipitation ': ' 0.00I',
 'relative_humidity': 56.1,
 'station': 948701,
 'windspeed_knots': 5.1}
{'GHI_w/m2': 140,
 '_id': ObjectId('60af1c5e9343690fa9450716'),
 'air_temperature_celcius': 16,
 'date': '04/04/2018',
 'fires': {'_id': ObjectId('60af1c689343690fa9451174'),
           'confidence': 100,
           'date': '04/04/2018',
           'datetime': '2018-04-04T04:32:50',
           'latitude': -36.343,
           'longitude': 142.1986,
           'surface_temperature_celcius': 123},
 'max_wind_speed': 12.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 47

**f. Find the number of fires each day. You are required to only display the total number of fires and the date in the output.**

In [25]:
results = climates.aggregate(
    [
        {"$project": 
             { 
                 "date": 1,
                 "fire_count": { "$size":"$fires" },
                 "_id": 0
                 
             }
        }
    ]
)

for document in results:
    pprint(document)

{'date': '31/12/2017', 'fire_count': 0}
{'date': '02/01/2018', 'fire_count': 0}
{'date': '03/01/2018', 'fire_count': 0}
{'date': '04/01/2018', 'fire_count': 0}
{'date': '05/01/2018', 'fire_count': 0}
{'date': '06/01/2018', 'fire_count': 0}
{'date': '07/01/2018', 'fire_count': 0}
{'date': '08/01/2018', 'fire_count': 0}
{'date': '09/01/2018', 'fire_count': 0}
{'date': '10/01/2018', 'fire_count': 0}
{'date': '11/01/2018', 'fire_count': 0}
{'date': '12/01/2018', 'fire_count': 0}
{'date': '13/01/2018', 'fire_count': 0}
{'date': '14/01/2018', 'fire_count': 0}
{'date': '15/01/2018', 'fire_count': 0}
{'date': '16/01/2018', 'fire_count': 0}
{'date': '17/01/2018', 'fire_count': 0}
{'date': '18/01/2018', 'fire_count': 0}
{'date': '19/01/2018', 'fire_count': 0}
{'date': '20/01/2018', 'fire_count': 0}
{'date': '21/01/2018', 'fire_count': 0}
{'date': '22/01/2018', 'fire_count': 0}
{'date': '23/01/2018', 'fire_count': 0}
{'date': '24/01/2018', 'fire_count': 0}
{'date': '25/01/2018', 'fire_count': 0}


**g. Find the records of fires where the confidence is below 70.**

In [26]:
results = fires.find({"confidence":{"$lt":70}})

for document in results:
    pprint(document)

{'_id': ObjectId('60af1c679343690fa9450829'),
 'confidence': 67,
 'date': '27/12/2018',
 'datetime': '2018-12-27T00:02:15',
 'latitude': -35.554,
 'longitude': 143.307,
 'surface_temperature_celcius': 53}
{'_id': ObjectId('60af1c679343690fa945082c'),
 'confidence': 65,
 'date': '24/12/2018',
 'datetime': '2018-12-24T13:12:01',
 'latitude': -35.646,
 'longitude': 142.282,
 'surface_temperature_celcius': 32}
{'_id': ObjectId('60af1c679343690fa945082f'),
 'confidence': 69,
 'date': '16/12/2018',
 'datetime': '2018-12-16T15:38:39',
 'latitude': -37.624,
 'longitude': 149.332,
 'surface_temperature_celcius': 33}
{'_id': ObjectId('60af1c679343690fa9450830'),
 'confidence': 62,
 'date': '16/12/2018',
 'datetime': '2018-12-16T04:35:15',
 'latitude': -37.82,
 'longitude': 142.323,
 'surface_temperature_celcius': 54}
{'_id': ObjectId('60af1c679343690fa9450834'),
 'confidence': 59,
 'date': '16/12/2018',
 'datetime': '2018-12-16T04:34:57',
 'latitude': -37.644,
 'longitude': 149.346,
 'surface_te

 'latitude': -36.4437,
 'longitude': 143.5726,
 'surface_temperature_celcius': 42}
{'_id': ObjectId('60af1c689343690fa94509df'),
 'confidence': 59,
 'date': '15/05/2018',
 'datetime': '2018-05-15T04:34:50',
 'latitude': -36.3987,
 'longitude': 142.6332,
 'surface_temperature_celcius': 40}
{'_id': ObjectId('60af1c689343690fa94509e3'),
 'confidence': 69,
 'date': '15/05/2018',
 'datetime': '2018-05-15T04:33:20',
 'latitude': -36.3065,
 'longitude': 141.4495,
 'surface_temperature_celcius': 44}
{'_id': ObjectId('60af1c689343690fa94509e4'),
 'confidence': 66,
 'date': '15/05/2018',
 'datetime': '2018-05-15T04:33:20',
 'latitude': -37.5074,
 'longitude': 142.976,
 'surface_temperature_celcius': 43}
{'_id': ObjectId('60af1c689343690fa94509e7'),
 'confidence': 55,
 'date': '15/05/2018',
 'datetime': '2018-05-15T04:32:50',
 'latitude': -37.2622,
 'longitude': 144.3704,
 'surface_temperature_celcius': 39}
{'_id': ObjectId('60af1c689343690fa94509e8'),
 'confidence': 69,
 'date': '15/05/2018',
 '

 'surface_temperature_celcius': 40}
{'_id': ObjectId('60af1c689343690fa9450b49'),
 'confidence': 59,
 'date': '05/05/2018',
 'datetime': '2018-05-05T03:52:30',
 'latitude': -37.5406,
 'longitude': 142.9301,
 'surface_temperature_celcius': 46}
{'_id': ObjectId('60af1c689343690fa9450b4b'),
 'confidence': 57,
 'date': '05/05/2018',
 'datetime': '2018-05-05T03:50:40',
 'latitude': -35.9839,
 'longitude': 143.6719,
 'surface_temperature_celcius': 47}
{'_id': ObjectId('60af1c689343690fa9450b4d'),
 'confidence': 52,
 'date': '05/05/2018',
 'datetime': '2018-05-05T03:50:20',
 'latitude': -36.2542,
 'longitude': 143.6256,
 'surface_temperature_celcius': 38}
{'_id': ObjectId('60af1c689343690fa9450b53'),
 'confidence': 53,
 'date': '05/05/2018',
 'datetime': '2018-05-05T03:50:20',
 'latitude': -36.3596,
 'longitude': 141.0646,
 'surface_temperature_celcius': 38}
{'_id': ObjectId('60af1c689343690fa9450b56'),
 'confidence': 64,
 'date': '05/05/2018',
 'datetime': '2018-05-05T03:50:20',
 'latitude':

 'date': '18/04/2018',
 'datetime': '2018-04-18T04:44:50',
 'latitude': -37.8643,
 'longitude': 143.5099,
 'surface_temperature_celcius': 42}
{'_id': ObjectId('60af1c689343690fa9450d60'),
 'confidence': 68,
 'date': '18/04/2018',
 'datetime': '2018-04-18T04:44:50',
 'latitude': -37.9255,
 'longitude': 143.1236,
 'surface_temperature_celcius': 44}
{'_id': ObjectId('60af1c689343690fa9450d61'),
 'confidence': 63,
 'date': '18/04/2018',
 'datetime': '2018-04-18T04:44:50',
 'latitude': -37.8857,
 'longitude': 143.2777,
 'surface_temperature_celcius': 41}
{'_id': ObjectId('60af1c689343690fa9450d65'),
 'confidence': 52,
 'date': '18/04/2018',
 'datetime': '2018-04-18T04:44:50',
 'latitude': -37.8375,
 'longitude': 143.4153,
 'surface_temperature_celcius': 39}
{'_id': ObjectId('60af1c689343690fa9450d66'),
 'confidence': 57,
 'date': '18/04/2018',
 'datetime': '2018-04-18T04:44:50',
 'latitude': -37.8635,
 'longitude': 143.1986,
 'surface_temperature_celcius': 39}
{'_id': ObjectId('60af1c689343

 'longitude': 145.1698,
 'surface_temperature_celcius': 45}
{'_id': ObjectId('60af1c689343690fa9450f85'),
 'confidence': 68,
 'date': '13/04/2018',
 'datetime': '2018-04-13T04:26:30',
 'latitude': -37.1949,
 'longitude': 142.675,
 'surface_temperature_celcius': 44}
{'_id': ObjectId('60af1c689343690fa9450f8c'),
 'confidence': 62,
 'date': '13/04/2018',
 'datetime': '2018-04-13T04:26:30',
 'latitude': -37.4784,
 'longitude': 143.015,
 'surface_temperature_celcius': 47}
{'_id': ObjectId('60af1c689343690fa9450f8e'),
 'confidence': 66,
 'date': '13/04/2018',
 'datetime': '2018-04-13T04:26:30',
 'latitude': -37.5358,
 'longitude': 143.0572,
 'surface_temperature_celcius': 43}
{'_id': ObjectId('60af1c689343690fa9450f8f'),
 'confidence': 59,
 'date': '13/04/2018',
 'datetime': '2018-04-13T04:26:30',
 'latitude': -37.5077,
 'longitude': 143.2495,
 'surface_temperature_celcius': 40}
{'_id': ObjectId('60af1c689343690fa9450f90'),
 'confidence': 57,
 'date': '13/04/2018',
 'datetime': '2018-04-13T0

 'datetime': '2018-04-01T04:02:40',
 'latitude': -36.2044,
 'longitude': 145.7706,
 'surface_temperature_celcius': 51}
{'_id': ObjectId('60af1c689343690fa94511de'),
 'confidence': 57,
 'date': '01/04/2018',
 'datetime': '2018-04-01T04:02:40',
 'latitude': -36.5245,
 'longitude': 143.1556,
 'surface_temperature_celcius': 43}
{'_id': ObjectId('60af1c689343690fa94511df'),
 'confidence': 50,
 'date': '01/04/2018',
 'datetime': '2018-04-01T04:02:40',
 'latitude': -36.1002,
 'longitude': 147.6494,
 'surface_temperature_celcius': 39}
{'_id': ObjectId('60af1c689343690fa94511e9'),
 'confidence': 68,
 'date': '31/03/2018',
 'datetime': '2018-03-31T04:57:50',
 'latitude': -36.1649,
 'longitude': 145.9352,
 'surface_temperature_celcius': 44}
{'_id': ObjectId('60af1c689343690fa94511ea'),
 'confidence': 68,
 'date': '31/03/2018',
 'datetime': '2018-03-31T04:57:40',
 'latitude': -36.0436,
 'longitude': 145.7964,
 'surface_temperature_celcius': 44}
{'_id': ObjectId('60af1c689343690fa94511f0'),
 'confi

**h. Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output.**

In [27]:

results = climates.aggregate(
    [

        {
            "$lookup":{
                    "from": "fire_historic",
                    "localField": "fires",
                    "foreignField" : "_id",
                    "as": "fires"
            }
        },
        {
            "$unwind": {
                 "path": "$fires",
                 "preserveNullAndEmptyArrays": True
                 
             }
        },
        {
            "$group": {
                "_id": "$date",
                "avg_surface_temp": {
                    "$avg": "$fires.surface_temperature_celcius"
                }
            }
        },
        { 
            "$project": {  
                        "_id": 0,
                        "date": "$_id",
                        "avg_surface_temp": 1
            }
        }
        
    ]
)

for document in results:
    pprint(document)


{'avg_surface_temp': None, 'date': '31/12/2018'}
{'avg_surface_temp': 62.75, 'date': '27/12/2018'}
{'avg_surface_temp': 54.0, 'date': '25/12/2018'}
{'avg_surface_temp': None, 'date': '23/12/2018'}
{'avg_surface_temp': None, 'date': '19/12/2018'}
{'avg_surface_temp': 60.0, 'date': '13/12/2018'}
{'avg_surface_temp': 44.0, 'date': '12/12/2018'}
{'avg_surface_temp': 50.6, 'date': '08/12/2018'}
{'avg_surface_temp': None, 'date': '07/12/2018'}
{'avg_surface_temp': None, 'date': '06/12/2018'}
{'avg_surface_temp': None, 'date': '03/12/2018'}
{'avg_surface_temp': None, 'date': '20/12/2018'}
{'avg_surface_temp': None, 'date': '02/12/2018'}
{'avg_surface_temp': 42.0, 'date': '28/11/2018'}
{'avg_surface_temp': None, 'date': '26/11/2018'}
{'avg_surface_temp': None, 'date': '20/11/2018'}
{'avg_surface_temp': None, 'date': '17/11/2018'}
{'avg_surface_temp': None, 'date': '16/11/2018'}
{'avg_surface_temp': 60.625, 'date': '29/11/2018'}
{'avg_surface_temp': None, 'date': '15/11/2018'}
{'avg_surface_tem

**i. Find the top 10 records with the lowest GHI.**

In [28]:
results = climates.aggregate(
    [

        {
            "$lookup":{
                    "from": "fire_historic",
                    "localField": "fires",
                    "foreignField" : "_id",
                    "as": "fires"
            }
        },
        {
            "$sort": {"GHI_w/m2": 1}
        },
        {
            "$limit": 10
        }
    ]
)

for document in results:
    pprint(document)

{'GHI_w/m2': 47,
 '_id': ObjectId('60af1c5e9343690fa945078e'),
 'air_temperature_celcius': 5,
 'date': '02/08/2018',
 'fires': [{'_id': ObjectId('60af1c689343690fa9450949'),
            'confidence': 94,
            'date': '02/08/2018',
            'datetime': '2018-08-02T03:45:40',
            'latitude': -37.4796,
            'longitude': 141.9403,
            'surface_temperature_celcius': 87},
           {'_id': ObjectId('60af1c689343690fa945094a'),
            'confidence': 54,
            'date': '02/08/2018',
            'datetime': '2018-08-02T03:45:00',
            'latitude': -37.491,
            'longitude': 141.936,
            'surface_temperature_celcius': 40}],
 'max_wind_speed': 5.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 38.6,
 'station': 948701,
 'windspeed_knots': 1.8}
{'GHI_w/m2': 48,
 '_id': ObjectId('60af1c5e9343690fa945076d'),
 'air_temperature_celcius': 5,
 'date': '30/06/2018',
 'fires': [{'_id': ObjectId('60af1c689343690fa9450964'),
            'c

**j. Find the records with a 24-hour precipitation recorded between 0.20 to 0.35.**

In [29]:
results = climates.aggregate(
    [
        {
            "$match": {
                        "$or" : [ 
                                { "precipitation ": { "$regex": "^ 0\.[2][0-9]" }},
                                { "precipitation ": { "$regex": "^ 0\.[3][0-5]" }}
                        ],
                        "precipitation ": { "$regex": "G$" },
                        
            }
        },
        {
            "$lookup":{
                    "from": "fire_historic",
                    "localField": "fires",
                    "foreignField" : "_id",
                    "as": "fires"
            }
        }
    ]
)

for document in results:
    pprint(document)
    
    
   

{'GHI_w/m2': 157,
 '_id': ObjectId('60af1c5e9343690fa94506c5'),
 'air_temperature_celcius': 19,
 'date': '13/01/2018',
 'fires': [],
 'max_wind_speed': 18.1,
 'precipitation ': ' 0.31G',
 'relative_humidity': 54.1,
 'station': 948700,
 'windspeed_knots': 11.2}
{'GHI_w/m2': 146,
 '_id': ObjectId('60af1c5e9343690fa9450710'),
 'air_temperature_celcius': 17,
 'date': '29/03/2018',
 'fires': [{'_id': ObjectId('60af1c689343690fa94511f6'),
            'confidence': 69,
            'date': '29/03/2018',
            'datetime': '2018-03-29T00:48:40',
            'latitude': -34.2648,
            'longitude': 141.6325,
            'surface_temperature_celcius': 51}],
 'max_wind_speed': 21.0,
 'precipitation ': ' 0.24G',
 'relative_humidity': 49.9,
 'station': 948701,
 'windspeed_knots': 12.2}
{'GHI_w/m2': 166,
 '_id': ObjectId('60af1c5e9343690fa9450726'),
 'air_temperature_celcius': 20,
 'date': '20/04/2018',
 'fires': [{'_id': ObjectId('60af1c689343690fa9450c6a'),
            'confidence': 84,
